In [1]:
import os
import itertools
import numpy as np
import pandas as pd

In [18]:
def get_df(path):
    df = pd.read_csv(path, index_col='Unnamed: 0')
    return df

def get_val_nll(df):
    return df.val_or_test_nll.values[-1]

def get_val_loss(df):
    return df.val_or_test_loss.values[-1]

def get_val_acc(df):
    return df.val_or_test_acc.values[-1]

def get_last_epoch(df):
    return df.iloc[-1]

def get_nonlearned_hyperparameters(experiments_path, lr_0s, ns, random_states, weight_decays):
    columns = ['lr_0', 'n', 'random_state', 'val_acc', 'val_loss', 'val_nll', 'weight_decay']
    df = pd.DataFrame(columns=columns)
    for n, random_state in itertools.product(ns, random_states):
        best_val_loss = np.inf
        best_val_nll = np.inf
        best_hyperparameters = None
        for lr_0, weight_decay in itertools.product(lr_0s, weight_decays):
            model_name = 'nonlearned_lr_0={}_n={}_random_state={}_weight_decay={}'\
            .format(lr_0, n, random_state, weight_decay)
            path =  '{}/{}.csv'.format(experiments_path, model_name)
            val_loss = get_val_loss(get_df(path))
            val_nll = get_val_nll(get_df(path))
            val_acc = get_val_acc(get_df(path))
            #if val_loss < best_val_loss: best_val_loss = val_loss; best_hyperparameters = [lr_0, n, random_state, val_acc, val_loss, val_nll, weight_decay]
            if val_nll < best_val_nll: best_val_nll = val_nll; best_hyperparameters = [lr_0, n, random_state, val_acc, val_loss, val_nll, weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters
    return df

def get_adapted_hyperparameters(experiments_path, lr_0s, ns, random_states, weight_decays):
    columns = ['lr_0', 'n', 'random_state', 'val_acc', 'val_loss', 'val_nll', 'weight_decay']
    df = pd.DataFrame(columns=columns)
    for n, random_state in itertools.product(ns, random_states):
        best_val_loss = np.inf
        best_val_nll = np.inf
        best_hyperparameters = None
        for lr_0, weight_decay in itertools.product(lr_0s, weight_decays):
            model_name = 'adapted_lr_0={}_n={}_random_state={}_weight_decay={}'\
            .format(lr_0, n, random_state, weight_decay)
            path =  '{}/{}.csv'.format(experiments_path, model_name)
            val_loss = get_val_loss(get_df(path))
            val_nll = get_val_nll(get_df(path))
            val_acc = get_val_acc(get_df(path))
            #if val_loss < best_val_loss: best_val_loss = val_loss; best_hyperparameters = [lr_0, n, random_state, val_acc, val_loss, val_nll, weight_decay]
            if val_nll < best_val_nll: best_val_nll = val_nll; best_hyperparameters = [lr_0, n, random_state, val_acc, val_loss, val_nll, weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters
    return df

def get_learned_hyperparameters(experiments_path, lr_0s, ns, prior_scales, random_states, weight_decays):
    columns = ['lr_0', 'n', 'prior_scale', 'random_state', 'val_acc', 'val_loss', 'val_nll', 'weight_decay']
    df = pd.DataFrame(columns=columns)
    for n, prior_scale, random_state in itertools.product(ns, prior_scales, random_states):
        best_val_loss = np.inf
        best_val_nll = np.inf
        best_hyperparameters = None
        for lr_0, weight_decay in itertools.product(lr_0s, weight_decays):
            model_name = 'learned_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
            .format(lr_0, n, prior_scale, random_state, weight_decay)
            path =  '{}/{}.csv'.format(experiments_path, model_name)
            val_loss = get_val_loss(get_df(path))
            val_acc = get_val_acc(get_df(path))
            val_nll = get_val_nll(get_df(path))
            #if val_loss < best_val_loss: best_val_loss = val_loss; best_hyperparameters = [lr_0, n, prior_scale, random_state, val_acc, val_loss, val_nll, weight_decay]
            if val_nll < best_val_nll: best_val_nll = val_nll; best_hyperparameters = [lr_0, n, prior_scale, random_state, val_acc, val_loss, val_nll, weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters
    return df


In [3]:
def get_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays):
    columns = ['lr_0', 'n', 'prior_scale', 'prior_type', 'random_state', 'val_acc', 'weight_decay']
    df = pd.DataFrame(columns=columns)
    for n, prior_scale, random_state in itertools.product(ns, prior_scales, random_states):
        best_val_nll = np.inf
        best_hyperparameters = None
        for lr_0, weight_decay in itertools.product(lr_0s, weight_decays):
            if prior_scale:
                model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, prior_scale, random_state, weight_decay)
            else:
                model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, random_state, weight_decay)
            path =  '{}/{}.csv'.format(experiments_path, model_name)
            val_nll = get_val_nll(get_df(path))
            val_acc = get_val_acc(get_df(path))
            if val_nll < best_val_nll: best_val_nll = val_nll; best_hyperparameters = [lr_0, n, prior_scale, prior_type, random_state, val_acc, weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters
    return df

def get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays,add_name=''):
    columns = ['lr_0', 'n', 'prior_scale', 'prior_type', 'random_state', 'val_acc', 'weight_decay']
    df = pd.DataFrame(columns=columns)
    for n, random_state in itertools.product(ns, random_states):
        best_val_nll = np.inf
        best_hyperparameters = None
        for lr_0, prior_scale, weight_decay in itertools.product(lr_0s, prior_scales, weight_decays):
            if prior_scale:
                model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, prior_scale, random_state, weight_decay) + add_name
            else:
                model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
                .format(prior_type, lr_0, n, random_state, weight_decay) + add_name
            path =  '{}/{}.csv'.format(experiments_path, model_name)
            val_nll = get_val_nll(get_df(path))
            val_acc = get_val_acc(get_df(path))
            if val_nll < best_val_nll: best_val_nll = val_nll; best_hyperparameters = [lr_0, n, prior_scale, prior_type, random_state, val_acc, weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters
    return df

def get_results(df, experiments_path,add_name=''):
    columns = ['n', 'prior_scale', 'prior_type', 'random_state', 'test_acc', 'test_loss', 'test_nll', 
           'test_prior', 'train_acc', 'train_loss', 'train_nll', 'train_prior']
    results = pd.DataFrame(columns=columns)
    for row_index, row in df.iterrows():
        if row.prior_scale:
            model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
            .format(row.prior_type, row.lr_0, row.n, row.prior_scale, row.random_state, row.weight_decay) + add_name
        else:
            model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
            .format(row.prior_type, row.lr_0, row.n, row.random_state, row.weight_decay) + add_name
        path =  '{}/{}.csv'.format(experiments_path, model_name)
        last_epoch = get_last_epoch(get_df(path))
        results_row = [int(row.n), row.prior_scale, row.prior_type, int(row.random_state), 
                       last_epoch.val_or_test_acc, last_epoch.val_or_test_loss, 
                       last_epoch.val_or_test_nll, last_epoch.val_or_test_prior, 
                       last_epoch.train_acc, last_epoch.train_loss, 
                       last_epoch.train_nll, last_epoch.train_prior]
        results.loc[results.shape[0]] = results_row
    return results

### Nonlearned

In [4]:
dataset_path = '../oxpets_dataset'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_OXPETS'
lr_0s = np.logspace(-1, -4, num=4)
ns = [3441, 370, 37]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
random_states = [1001, 2001, 3001]
prior_type = 'nonlearned'
wandb_project = 'tuned_OXPETS'
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
count = -1
for lr_0, n, random_state, weight_decay in itertools.product(lr_0s, ns, random_states, weight_decays):
    count += 1
    model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
    .format(prior_type, lr_0, n, random_state, weight_decay)
#     print('    "python ../src/OXPETS_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --tune --wandb --wandb_project=\'{}\' --weight_decay={}"'\
#           .format(dataset_path, experiments_path, lr_0, model_name, n, prior_path, prior_type, random_state, wandb_project, weight_decay))
print(count)

215


In [5]:
####### Find the best hypers for nonlearned
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_OXPETS'
lr_0s = np.logspace(-1, -4, num=4)
prior_scales = [None]
prior_type = 'nonlearned'
ns = [3441, 370, 37]
random_states = [1001, 2001, 3001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
nonlearned_hyperparameters = get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays)
nonlearned_hyperparameters

,lr_0,n,prior_scale,prior_type,random_state,val_acc,weight_decay
0,0.0100,3441,None,nonlearned,1001,0.897898,0.001000
1,0.0100,3441,None,nonlearned,2001,0.897898,0.001000
2,0.0100,3441,None,nonlearned,3001,0.911411,0.000001
3,0.0100,370,None,nonlearned,1001,0.689189,0.000010
4,0.0010,370,None,nonlearned,2001,0.635135,0.000001
5,0.0001,370,None,nonlearned,3001,0.554054,0.000000
6,0.0010,37,None,nonlearned,1001,0.135135,0.010000
7,0.0010,37,None,nonlearned,2001,0.135135,0.010000
8,0.0010,37,None,nonlearned,3001,0.162162,0.010000


In [11]:
####### Make new run commands with the best hypers
dataset_path = '../oxpets_dataset'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_OXPETS'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'nonlearned'
wandb_project = 'retrained_OXPETS'

count = -1
for row_index, row in nonlearned_hyperparameters.iterrows():
    count += 1
    model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
    .format(prior_type, row.lr_0, int(row.n), int(row.random_state), row.weight_decay)
#     print('    "python ../src/OXPETS_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --wandb --wandb_project=\'{}\' --weight_decay={}"'\
#           .format(dataset_path, experiments_path, row.lr_0, model_name, int(row.n), prior_path, prior_type, int(row.random_state), wandb_project, row.weight_decay))
print(count)

8


In [24]:
##### nonlearned results
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_OXPETS'
results = get_results(nonlearned_hyperparameters, experiments_path)
results

,n,prior_scale,prior_type,random_state,test_acc,test_loss,test_nll,test_prior,train_acc,train_loss,train_nll,train_prior
0,3441,None,nonlearned,1001,0.864874,0.555606,0.555606,0.0,1.000000,0.000219,0.000219,0.0
1,3441,None,nonlearned,2001,0.866187,0.547540,0.547540,0.0,1.000000,0.000212,0.000212,0.0
2,3441,None,nonlearned,3001,0.859838,0.655502,0.655502,0.0,1.000000,0.000021,0.000021,0.0
3,370,None,nonlearned,1001,0.575321,1.720191,1.720191,0.0,1.000000,0.000037,0.000037,0.0
4,370,None,nonlearned,2001,0.537177,1.828269,1.828269,0.0,1.000000,0.000633,0.000633,0.0
5,370,None,nonlearned,3001,0.532070,1.782722,1.782722,0.0,0.991892,0.156752,0.156752,0.0
6,37,None,nonlearned,1001,0.172697,3.327513,3.327513,0.0,1.000000,0.003454,0.003454,0.0
7,37,None,nonlearned,2001,0.145260,3.398588,3.398588,0.0,1.000000,0.003515,0.003515,0.0
8,37,None,nonlearned,3001,0.200102,3.250308,3.250308,0.0,1.000000,0.003434,0.003434,0.0


In [9]:
results.groupby(['n']).mean()

,test_acc,test_loss,test_nll,test_prior,train_acc,train_loss,train_nll,train_prior
n,,,,,,,,
37,0.172686,3.325470,3.325470,0.0,1.000000,0.003468,0.003468,0.0
370,0.548189,1.777061,1.777061,0.0,0.997297,0.052474,0.052474,0.0
3441,0.863633,0.586216,0.586216,0.0,1.000000,0.000151,0.000151,0.0


### Adapted

In [12]:
dataset_path = '../oxpets_dataset'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_OXPETS'
lr_0s = np.logspace(-1, -4, num=4)
ns = [3441, 370, 37]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
random_states = [1001, 2001, 3001]
prior_type = 'adapted'
wandb_project = 'tuned_OXPETS'
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
count = -1
for lr_0, n, random_state, weight_decay in itertools.product(lr_0s, ns, random_states, weight_decays):
    count += 1
    model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
    .format(prior_type, lr_0, n, random_state, weight_decay)
#     print('    "python ../src/OXPETS_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --tune --wandb --wandb_project=\'{}\' --weight_decay={}"'\
#           .format(dataset_path, experiments_path, lr_0, model_name, n, prior_path, prior_type, random_state, wandb_project, weight_decay))
print(count)

215


In [13]:
####### Find the best hypers for nonlearned
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_OXPETS'
lr_0s = np.logspace(-1, -4, num=4)
prior_scales = [None]
prior_type = 'adapted'
ns = [3441, 370, 37]
random_states = [1001, 2001, 3001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
adapted_hyperparameters = get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays)
adapted_hyperparameters

,lr_0,n,prior_scale,prior_type,random_state,val_acc,weight_decay
0,0.0100,3441,None,adapted,1001,0.896396,0.001000
1,0.0100,3441,None,adapted,2001,0.899399,0.001000
2,0.0100,3441,None,adapted,3001,0.920420,0.001000
3,0.0100,370,None,adapted,1001,0.702703,0.000010
4,0.1000,370,None,adapted,2001,0.540541,0.010000
5,0.1000,370,None,adapted,3001,0.675676,0.010000
6,0.0001,37,None,adapted,1001,0.135135,0.000010
7,0.0001,37,None,adapted,2001,0.135135,0.000001
8,0.0001,37,None,adapted,3001,0.162162,0.000001


In [15]:
####### Make new run commands with the best hypers
dataset_path = '../oxpets_dataset'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_OXPETS'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'adapted'
wandb_project = 'retrained_OXPETS'


count = -1
for row_index, row in adapted_hyperparameters.iterrows():
    count += 1
    model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}'\
    .format(prior_type, row.lr_0, int(row.n), int(row.random_state), row.weight_decay)
#     print('    "python ../src/OXPETS_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --wandb --wandb_project=\'{}\' --weight_decay={}"'\
#           .format(dataset_path, experiments_path, row.lr_0, model_name, int(row.n), prior_path, prior_type, int(row.random_state), wandb_project, row.weight_decay))
print(count)

8


In [16]:
##### nonlearned results
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_OXPETS'
results = get_results(adapted_hyperparameters, experiments_path)
results

,n,prior_scale,prior_type,random_state,test_acc,test_loss,test_nll,test_prior,train_acc,train_loss,train_nll,train_prior
0,3441,None,adapted,1001,0.872254,0.566250,0.534827,0.031423,1.0,0.031560,0.000138,0.031423
1,3441,None,adapted,2001,0.863352,0.583711,0.554833,0.028878,1.0,0.029020,0.000142,0.028878
2,3441,None,adapted,3001,0.864451,0.596398,0.568093,0.028305,1.0,0.028443,0.000138,0.028305
3,370,None,adapted,1001,0.573260,1.722578,1.722463,0.000115,1.0,0.000151,0.000036,0.000115
4,370,None,adapted,2001,0.554725,1.679727,1.673891,0.005836,1.0,0.006635,0.000798,0.005836
5,370,None,adapted,3001,0.567416,1.806014,1.800529,0.005485,1.0,0.006123,0.000638,0.005485
6,37,None,adapted,1001,0.181212,3.276047,3.276021,0.000026,1.0,0.014836,0.014810,0.000026
7,37,None,adapted,2001,0.151471,3.341888,3.341885,0.000003,1.0,0.014935,0.014932,0.000003
8,37,None,adapted,3001,0.207152,3.184841,3.184838,0.000003,1.0,0.015300,0.015298,0.000003


In [17]:
results.groupby(['n']).mean()

,test_acc,test_loss,test_nll,test_prior,train_acc,train_loss,train_nll,train_prior
n,,,,,,,,
37,0.179945,3.267592,3.267581,0.000010,1.0,0.015024,0.015013,0.000010
370,0.565134,1.736106,1.732294,0.003812,1.0,0.004303,0.000491,0.003812
3441,0.866686,0.582119,0.552584,0.029535,1.0,0.029674,0.000139,0.029535


### Adapted corrected

In [ ]:
dataset_path = '../oxpets_dataset'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_OXPETS'
lr_0s = np.logspace(-1, -4, num=4)
ns = [3441, 370, 37]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
random_states = [1001, 2001, 3001]
prior_type = 'adapted'
wandb_project = 'tuned_OXPETS'
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
count = -1
for lr_0, n, random_state, weight_decay in itertools.product(lr_0s, ns, random_states, weight_decays):
    count += 1
    model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}_corrected'\
    .format(prior_type, lr_0, n, random_state, weight_decay)
#     print('    "python ../src/OXPETS_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --tune --wandb --wandb_project=\'{}\' --weight_decay={}"'\
#           .format(dataset_path, experiments_path, lr_0, model_name, n, prior_path, prior_type, random_state, wandb_project, weight_decay))
print(count)

In [19]:
####### Find the best hypers for nonlearned
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_OXPETS'
lr_0s = np.logspace(-1, -4, num=4)
prior_scales = [None]
prior_type = 'adapted'
ns = [3441, 370, 37]
random_states = [1001, 2001, 3001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
adapted_hyperparameters = get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays,add_name='_corrected')
adapted_hyperparameters

,lr_0,n,prior_scale,prior_type,random_state,val_acc,weight_decay
0,0.0100,3441,None,adapted,1001,0.902402,0.0000
1,0.0100,3441,None,adapted,2001,0.905405,0.0010
2,0.0100,3441,None,adapted,3001,0.923423,0.0010
3,0.0100,370,None,adapted,1001,0.675676,0.0001
4,0.0100,370,None,adapted,2001,0.635135,0.0010
5,0.0001,370,None,adapted,3001,0.540541,0.0000
6,0.1000,37,None,adapted,1001,0.135135,0.0100
7,0.1000,37,None,adapted,2001,0.135135,0.0100
8,0.1000,37,None,adapted,3001,0.162162,0.0100


In [26]:
ns = [37]
random_states = [1001, 2001, 3001]

columns = ['lr_0', 'n', 'random_state', 'val_acc', 'val_loss', 'val_nll', 'weight_decay']
df = pd.DataFrame(columns=columns)
for n, random_state in itertools.product(ns, random_states):
    best_val_loss = np.inf
    best_val_nll = np.inf
    best_hyperparameters = None
    for lr_0, weight_decay in itertools.product(lr_0s, weight_decays):
        model_name = 'adapted_lr_0={}_n={}_random_state={}_weight_decay={}_corrected'\
        .format(lr_0, n, random_state, weight_decay)
        path =  '{}/{}.csv'.format(experiments_path, model_name)
        val_loss = get_val_loss(get_df(path))
        val_nll = get_val_nll(get_df(path))
        val_acc = get_val_acc(get_df(path))
        best_hyperparameters = [lr_0, n, random_state, val_acc, val_loss, val_nll, weight_decay]
        #if val_loss < best_val_loss: best_val_loss = val_loss; best_hyperparameters = [lr_0, n, random_state, val_acc, val_loss, val_nll, weight_decay]
        #if val_nll < best_val_nll: best_val_nll = val_nll; best_hyperparameters = [lr_0, n, random_state, val_acc, val_loss, val_nll, weight_decay]
        df.loc[df.shape[0]] = best_hyperparameters

In [27]:
df

,lr_0,n,random_state,val_acc,val_loss,val_nll,weight_decay
0,0.1000,37.0,1001.0,0.135135,1.058874,1.045218,0.010000
1,0.1000,37.0,1001.0,0.135135,1.103653,1.101044,0.001000
2,0.1000,37.0,1001.0,0.135135,1.482554,1.479962,0.000100
3,0.1000,37.0,1001.0,0.135135,1.565403,1.564960,0.000010
4,0.1000,37.0,1001.0,0.135135,1.599938,1.599892,0.000001
5,0.1000,37.0,1001.0,0.135135,1.638192,1.638192,0.000000
6,0.0100,37.0,1001.0,0.135135,1.108044,1.087820,0.010000
7,0.0100,37.0,1001.0,0.135135,1.273953,1.265679,0.001000
8,0.0100,37.0,1001.0,0.135135,1.310786,1.309538,0.000100
9,0.0100,37.0,1001.0,0.135135,1.313955,1.313824,0.000010


In [20]:
####### Make new run commands with the best hypers
dataset_path = '../oxpets_dataset'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_OXPETS'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'adapted'
wandb_project = 'retrained_OXPETS'


count = -1
for row_index, row in adapted_hyperparameters.iterrows():
    count += 1
    model_name = '{}_lr_0={}_n={}_random_state={}_weight_decay={}_corrected'\
    .format(prior_type, row.lr_0, int(row.n), int(row.random_state), row.weight_decay)
    print('    "python ../src/OXPETS_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --random_state={} --wandb --wandb_project=\'{}\' --weight_decay={}"'\
          .format(dataset_path, experiments_path, row.lr_0, model_name, int(row.n), prior_path, prior_type, int(row.random_state), wandb_project, row.weight_decay))
print(count)

    "python ../src/OXPETS_main.py --dataset_path='../oxpets_dataset' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_OXPETS' --lr_0=0.01 --model_name='adapted_lr_0=0.01_n=3441_random_state=1001_weight_decay=0.0_corrected' --n=3441 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='adapted' --random_state=1001 --wandb --wandb_project='retrained_OXPETS' --weight_decay=0.0"
    "python ../src/OXPETS_main.py --dataset_path='../oxpets_dataset' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_OXPETS' --lr_0=0.01 --model_name='adapted_lr_0=0.01_n=3441_random_state=2001_weight_decay=0.001_corrected' --n=3441 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='adapted' --random_state=2001 --wandb --wandb_project='retrained_OXPETS' --weight_decay=0.001"
    "python ../src/OXPETS_main.py --dataset_path='../oxpets_dataset' --experiments_pat

In [6]:
##### nonlearned results
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_OXPETS'
results = get_results(adapted_hyperparameters, experiments_path,add_name='_corrected')
results

,n,prior_scale,prior_type,random_state,test_acc,test_loss,test_nll,test_prior,train_acc,train_loss,train_nll,train_prior
0,3441,None,adapted,1001,0.854081,0.659033,0.659033,0.000000,1.000000,0.000024,0.000024,0.000000
1,3441,None,adapted,2001,0.867176,0.579947,0.544586,0.035361,1.000000,0.035516,0.000155,0.035361
2,3441,None,adapted,3001,0.869879,0.557198,0.523561,0.033637,1.000000,0.033801,0.000165,0.033637
3,370,None,adapted,1001,0.577203,1.711945,1.710178,0.001767,1.000000,0.001815,0.000048,0.001767
4,370,None,adapted,2001,0.554506,1.878757,1.866707,0.012050,1.000000,0.012261,0.000211,0.012050
5,370,None,adapted,3001,0.533362,1.778537,1.778537,0.000000,0.994595,0.140283,0.140283,0.000000
6,37,None,adapted,1001,0.070150,3.627272,3.612332,0.014940,1.000000,0.015848,0.000908,0.014940
7,37,None,adapted,2001,0.053620,3.703639,3.687906,0.015733,1.000000,0.016719,0.000986,0.015733
8,37,None,adapted,3001,0.084853,3.576377,3.560486,0.015891,1.000000,0.016958,0.001067,0.015891


In [12]:
pd.DataFrame([results.groupby(['n']).mean()['test_acc'],results.groupby(['n']).min()['test_acc'],results.groupby(['n']).max()['test_acc']],index=['mean','min','max'])

n,37,370,3441
mean,0.069541,0.555024,0.863712
min,0.053620,0.533362,0.854081
max,0.084853,0.577203,0.869879


### Learned

In [27]:
dataset_path = '../oxpets_dataset'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_OXPETS'
lr_0s = np.logspace(-1, -4, num=4)
ns = [3441, 370, 37]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
#prior_scales = np.logspace(0, 9, num=10)
prior_scales = [1e0]
random_states = [1001, 2001, 3001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
wandb_project = 'tuned_OXPETS'
prior_type = 'learned'
count = -1

for lr_0, n, random_state, weight_decay, prior_scale in itertools.product(lr_0s, ns, random_states, weight_decays,prior_scales):
    count += 1
    model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
    .format(prior_type, lr_0, n, prior_scale, random_state, weight_decay)
#     print('    "python ../src/OXPETS_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --prior_scale={} --random_state={} --tune --wandb --wandb_project=\'{}\' --weight_decay={}"'\
#           .format(dataset_path, experiments_path, lr_0, model_name, n, prior_path, prior_type, prior_scale, random_state, wandb_project, weight_decay))
print(count)

215


In [13]:
####### Find the best hypers for nonlearned
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tuned_OXPETS'
lr_0s = np.logspace(-1, -4, num=4)
#prior_scales = [1e0]
prior_scales = [1e0,10,100,1e3,1e4,1e5,1e6,1e7,1e8,1e9]
prior_type = 'learned'
ns = [3441, 370, 37]
random_states = [1001, 2001, 3001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
learned_hyperparameters = get_best_hyperparameters(experiments_path, lr_0s, ns, prior_scales, prior_type, random_states, weight_decays)
learned_hyperparameters

,lr_0,n,prior_scale,prior_type,random_state,val_acc,weight_decay
0,0.01,3441,100,learned,1001,0.917417,0.001000
1,0.01,3441,10,learned,2001,0.905405,0.000001
2,0.01,3441,1000,learned,3001,0.912913,0.000010
3,0.01,370,1e+09,learned,1001,0.689189,0.000010
4,0.10,370,1000,learned,2001,0.594595,0.000010
5,0.10,370,1,learned,3001,0.648649,0.000010
6,0.01,37,10000,learned,1001,0.135135,0.010000
7,0.01,37,10000,learned,2001,0.135135,0.010000
8,0.01,37,10000,learned,3001,0.162162,0.010000


In [44]:
####### Make new run commands with the best hypers
dataset_path = '../oxpets_dataset'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_OXPETS'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_type = 'learned'
wandb_project = 'retrained_OXPETS'

count = -1
for row_index, row in learned_hyperparameters.iterrows():
    count += 1
    model_name = '{}_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
    .format(prior_type, row.lr_0, int(row.n), row.prior_scale, int(row.random_state), row.weight_decay)   
    print('    "python ../src/OXPETS_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_type=\'{}\' --prior_scale={} --random_state={} --wandb --wandb_project=\'{}\' --weight_decay={}"'\
           .format(dataset_path, experiments_path, row.lr_0, model_name, int(row.n), prior_path, prior_type, prior_scale, int(row.random_state), wandb_project, row.weight_decay))

print(count)

    "python ../src/OXPETS_main.py --dataset_path='../oxpets_dataset' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_OXPETS' --lr_0=0.01 --model_name='learned_lr_0=0.01_n=3441_prior_scale=100_random_state=1001_weight_decay=0.001' --n=3441 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='learned' --prior_scale=1.0 --random_state=1001 --wandb --wandb_project='retrained_OXPETS' --weight_decay=0.001"
    "python ../src/OXPETS_main.py --dataset_path='../oxpets_dataset' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_OXPETS' --lr_0=0.01 --model_name='learned_lr_0=0.01_n=3441_prior_scale=10_random_state=2001_weight_decay=1e-06' --n=3441 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_type='learned' --prior_scale=1.0 --random_state=2001 --wandb --wandb_project='retrained_OXPETS' --weight_decay=1e-06"
    "python ../src/OXPETS_main.py -

In [14]:
##### nonlearned results
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_OXPETS'
results = get_results(learned_hyperparameters, experiments_path)
results

,n,prior_scale,prior_type,random_state,test_acc,test_loss,test_nll,test_prior,train_acc,train_loss,train_nll,train_prior
0,3441,100,learned,1001,0.859576,-1484.726880,0.559501,1485.286377,1.000000,-1583.700607,0.000442,1583.701050
1,3441,10,learned,2001,0.867458,-1484.936099,0.529720,1485.465820,1.000000,-1583.892207,0.000247,1583.892456
2,3441,1000,learned,3001,0.863576,-1484.912000,0.553703,1485.465698,1.000000,-1583.892081,0.000244,1583.892334
3,370,1e+09,learned,1001,0.556153,-1483.753706,1.735783,1485.489502,1.000000,-14730.434570,0.000877,14730.435547
4,370,1000,learned,2001,0.565806,-1483.884221,1.606492,1485.490723,1.000000,-14730.446289,0.000922,14730.447266
5,370,1,learned,3001,0.586414,-1483.831912,1.658925,1485.490845,1.000000,-14730.447266,0.000848,14730.448242
6,37,10000,learned,1001,0.025467,-1481.574507,3.818809,1485.393311,1.000000,-147294.640625,0.178541,147294.812500
7,37,10000,learned,2001,0.037370,-1481.639218,3.753354,1485.392578,0.918919,-147294.500000,0.253638,147294.750000
8,37,10000,learned,3001,0.033337,-1481.358470,4.034843,1485.393311,0.810811,-147294.218750,0.595780,147294.812500


In [17]:
pd.DataFrame([results.groupby(['n']).mean()['test_acc'],results.groupby(['n']).min()['test_acc'],results.groupby(['n']).max()['test_acc']],index=['mean','min','max'])

n,37,370,3441
mean,0.032058,0.569458,0.863536
min,0.025467,0.556153,0.859576
max,0.037370,0.586414,0.867458


In [3]:
np.logspace(0, 9, num=10)

array([1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06, 1.e+07,
       1.e+08, 1.e+09])